# Group Chat with Tools

This notebook explains how to set up a group chat where each agent has unique capabilities and is equipped with specialized tools to perform specific tasks.

## Installation
```bash
pip install -U ag2[openai]
```

> **Note:** If you have been using `autogen` or `pyautogen`, all you need to do is upgrade it using:  
> ```bash
> pip install -U autogen[openai]
> ```
> or  
> ```bash
> pip install -U pyautogen[openai]
> ```
> as `pyautogen`, `autogen`, and `ag2` are aliases for the same PyPI package.  

## Imports

In [ ]:
import random

from autogen import (
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    LLMConfig,
    UserProxyAgent,
    register_function,
)

## Agent Configuration
  
The `GroupChat` will contain three agents:
- `sales_agent` - Responsible for selling tickets.
- `cancellation_agent` - Handles ticket cancellations.
- `user_proxy` - Acts as an intermediary between the user and other agents.

In [ ]:
llm_config = LLMConfig.from_json(path="OAI_CONFIG_LIST").where(model=["gpt-4o-mini"])

sales_agent = ConversableAgent(
    name="SalesAgent",
    llm_config=llm_config,
)

cancellation_agent = ConversableAgent(
    name="CanelationAgent",
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
)

## Tools Registration

In AG2, tool usage follows two steps:
- An agent suggests a tool to use (via its LLM).
- Another agent executes the tool.

We will define two tools:
- `buy_airplane_ticket`: Suggested by `sales_agent` and executed by `user_proxy` after user verification.
- `cancel_airplane_ticket`: Suggested by `cancellation_agent` and executed by `user_proxy` after user verification.

In [ ]:
def buy_airplane_ticket(from_location: str, to_location: str, date: str) -> str:
    ticket_number = random.randint(1000, 9999)
    return f"""Your ticket from {from_location} to {to_location} on {date} has been booked.
Your ticket number is {ticket_number}.
Please keep this number for future reference.
"""


register_function(
    buy_airplane_ticket,
    caller=sales_agent,
    executor=user_proxy,
    description="Buy an airplane ticket",
)


def cancel_airplane_ticket(ticket_number: str) -> str:
    return f"Your ticket with ticket number {ticket_number} has been canceled"


register_function(
    cancel_airplane_ticket,
    caller=cancellation_agent,
    executor=user_proxy,
    description="Cancel an airplane ticket",
)

## Creating and Initiating the Group Chat

Now, let's create and start the `GroupChat` with the three agents.

In [ ]:
groupchat = GroupChat(
    agents=[user_proxy, cancellation_agent, sales_agent],
    speaker_selection_method="auto",
    messages=[],
)

manager = GroupChatManager(
    name="group_manager",
    groupchat=groupchat,
    llm_config=llm_config,
)


user_proxy.initiate_chat(
    recipient=manager,
    message="I need to buy a plane ticket from New York to Los Angeles on 12th of April 2025",
)